In [47]:
from scipy.io import netcdf
import os
import glob
import numpy as np
import pandas as pd


In [75]:

import numpy as np
from scipy.io import netcdf
import pandas as pd
import datetime
import random
import pickle
import matplotlib.pyplot as plt


In [87]:
del uuz, vvz, ttz

In [92]:
files_list = glob.glob("data/MAR/concat*.nc")
files_list = sorted(files_list)

ls = []
time_size = 0
locations = [(2,31,61), (2,17,27), (2,23,39)] # (k,j,i)



for num_wf, location in enumerate(locations):
    time = np.empty((0))
    
    uuz80 = np.empty((0))
    vvz80 = np.empty((0))
    ttz80 = np.empty((0))

    uuz100 = np.empty((0))
    vvz100 = np.empty((0))
    ttz100 = np.empty((0))
    
    for netCDF_file_path in files_list:
        new_netcdf = {}
        with netcdf.netcdf_file(netCDF_file_path, 'r') as f:
            k = f.variables['UUZ'][0,:,0,0].shape[0] - 1 # 80 m 
            j = location[1] - 1 # j wind farm
            i = location[2] - 1 # i wind farm

            for name, var in f.variables.items():
                # new_netcdf[name] = var[:].copy()
                # print("name", name)
                if name[:4] == "TIME":
                    key = name
                    time = np.concatenate((time, var[:].copy() ))
                    time_size += time.shape[0]
                    print(var[0], time_size, netCDF_file_path)
                
                if name == "UUZ":
                    # print(var[:,k,j,i].shape)
                    uuz80 = np.concatenate((uuz80, var[:,-2,j,i].copy()))
                    uuz100 = np.concatenate((uuz100, var[:,-1,j,i].copy()))
                elif name == "VVZ": 
                    vvz80 = np.concatenate((vvz80, var[:,-2,j,i].copy()))
                    vvz100 = np.concatenate((vvz100, var[:,-1,j,i].copy()))
                elif name == "TTZ":
                    ttz80 = np.concatenate((ttz80, var[:,-2,j,i].copy()))
                    ttz100 = np.concatenate((ttz100, var[:,-1,j,i].copy()))

            

            f.close()
            
    dico = {"TIME":time, "UUZ80":uuz80, "VVZ80":vvz80, "TTZ80": ttz80,"UUZ100":uuz100, "VVZ100":vvz100, "TTZ100": ttz100}
    df = pd.DataFrame(dico)
    df.to_csv(f"data/windFarm{num_wf}.csv", index=False)
    

2304008.0 6432 data/MAR/concat_20210118_20210402.nc
2412008.0 14784 data/MAR/concat_20210403_20210430.nc
2452328.0 40128 data/MAR/concat_20210501_20211024.nc
2707208.0 66144 data/MAR/concat_20211025_20211031.nc
2717288.0 93024 data/MAR/concat_20211101_20211109.nc
2730248.0 121920 data/MAR/concat_20211110_20211130.nc
2760488.0 153600 data/MAR/concat_20211201_20211229.nc
2802247.5 185472 data/MAR/concat_20211230_20211231.nc
2805128.0 220320 data/MAR/concat_20220101_20220131.nc
2304008.0 226752 data/MAR/concat_20210118_20210402.nc
2412008.0 235104 data/MAR/concat_20210403_20210430.nc
2452328.0 260448 data/MAR/concat_20210501_20211024.nc
2707208.0 286464 data/MAR/concat_20211025_20211031.nc
2717288.0 313344 data/MAR/concat_20211101_20211109.nc
2730248.0 342240 data/MAR/concat_20211110_20211130.nc
2760488.0 373920 data/MAR/concat_20211201_20211229.nc
2802247.5 405792 data/MAR/concat_20211230_20211231.nc
2805128.0 440640 data/MAR/concat_20220101_20220131.nc
2304008.0 447072 data/MAR/concat_2

In [82]:
dico = {"UUZ":uuz, "VVZ":vvz, "TTZ": ttz, "TIME":time}
df = pd.DataFrame(dico)

In [93]:
for i in range(df["TIME"].shape[0] - 1):
    if df["TIME"][i] > df["TIME"][i+1]:
        print(i)

In [84]:
df["TIME"].shape

(34848,)

In [79]:
convert_min_to_date(start_time='2016-09-01 00:00:00', elapsed_minutes=2452313)

datetime.datetime(2021, 4, 30, 23, 53)

In [80]:
convert_min_to_date(start_time='2016-09-01 00:00:00', elapsed_minutes=2450888)

datetime.datetime(2021, 4, 30, 0, 8)

In [72]:
df["TIME"][25439 - 2:25439 + 4]

25437    2707162.5
25438    2707177.5
25439    2707192.5
25440    2705768.0
25441    2705783.0
25442    2705798.0
Name: TIME, dtype: float64

In [77]:
convert_min_to_date(start_time='2016-09-01 00:00:00', elapsed_minutes=2707192.5)

datetime.datetime(2021, 10, 24, 23, 52)

In [76]:
convert_min_to_date(start_time='2016-09-01 00:00:00', elapsed_minutes=2705768.0)

datetime.datetime(2021, 10, 24, 0, 8)

In [95]:
convert_min_to_date(start_time='2016-09-01 00:00:00', elapsed_minutes=df["TIME"][-1:])

datetime.datetime(2022, 1, 31, 23, 52)

In [73]:
def convert_min_to_date(start_time, elapsed_minutes):
    """
    Method to convert a given number of minutes elapsed from a
    given start_time date into a datetime object
    e.g.: 
    convert_min_to_date(start_time='2016-09-01 00:00:00', elapsed_minutes=4.0)
    datetime.datetime(2016, 9, 1, 0, 4)

    args:
    -----
        start_time: String 
                    ISO format date time e.g. "2016-09-01 00:00:00"
        elapsed_minutes: int or float
                         number of minutes elapsed from start time
    return:
    -------
        now_date: Datetime object
                  corresponding to start_time + elapsed_minutes
    """
    start_time = datetime.datetime.fromisoformat(start_time)
    delta = datetime.timedelta(minutes=int(elapsed_minutes))
    
    now_date = start_time + delta
    
    return now_date

In [60]:
df["TIME"].shape[0]

105120

In [33]:
uuz.shape

(105120,)

In [34]:
ttz.shape

(105120,)

In [36]:
vvz.shape

(105120,)

In [23]:
with netcdf.netcdf_file("data/MAR/concat.nc", 'w') as f:
    f.createDimension('TIME', time_size)
    f.createDimension('X21_100', 80)
    f.createDimension('Y21_70', 50)
    f.createDimension('ZULEV', 2)
    
    time = f.createVariable('TIME', 'int32', ('TIME',))
    UUZ = f.createVariable("UUZ", "float", ('TIME', 'ZULEV', 'Y21_70', 'X21_100',) )
    VVZ = f.createVariable("VVZ", "float", ('TIME', 'ZULEV', 'Y21_70', 'X21_100',) )
    TTZ = f.createVariable("VVZ", "float", ('TIME', 'ZULEV', 'Y21_70', 'X21_100',) )

    time.units = b'minutes since 2016-09-01 00:00:00'
    VVZ.units = b'm/s'


    start = 0
    end = 0
    for i, new_netcdf in enumerate(ls):
        print(i, "over", len(ls))
        
        if i in [6,8]:
            key = 'TIME2'
            end += new_netcdf[key].shape[0]
        elif i in [3,4]:
            key = 'TIME1'
            end += new_netcdf[key].shape[0]
        else:
            key = 'TIME'
            end += new_netcdf[key].shape[0]   
        
        
        time[start:end] = new_netcdf[key]
        UUZ[start:end, :, :, :] = new_netcdf['UUZ'][:, -2:, :, :]
        VVZ[start:end, :, :, :] = new_netcdf['VVZ'][:, -2:, :, :]
        TTZ[start:end, :, :, :] = new_netcdf['TTZ'][:, -2:, :, :]
        start = end 
    
    
    f.close()


0 over 9
1 over 9
2 over 9
3 over 9
4 over 9
5 over 9
6 over 9
7 over 9
8 over 9


In [8]:
files_list[1]

'data/MAR/concat_20210403_20210430.nc'

In [7]:
files_list[2]

'data/MAR/concat_20210430_20211024.nc'

In [9]:
files_list[-2:]

['data/MAR/concat_20211230_20211231.nc',
 'data/MAR/concat_20220101_20220131.nc']

In [11]:
files_list

['data/MAR/concat_20210118_20210402.nc',
 'data/MAR/concat_20210403_20210430.nc',
 'data/MAR/concat_20210430_20211024.nc',
 'data/MAR/concat_20211024_20211031.nc',
 'data/MAR/concat_20211101_20211109.nc',
 'data/MAR/concat_20211110_20211130.nc',
 'data/MAR/concat_20211201_20211229.nc',
 'data/MAR/concat_20211230_20211231.nc',
 'data/MAR/concat_20220101_20220131.nc']

In [22]:
for i, new_netcdf in enumerate(ls):
    # print(i, "over", len(ls))
    try:
        new_netcdf['TIME']        
        # print(i, "is ok")
    except:
        print(i, "is not ok ")
        print(new_netcdf.keys())

3 is not ok 
dict_keys(['X21_100', 'Y21_70', 'ZUVLEV', 'ZUVLEV_bnds', 'TIME1', 'UUZ', 'VVZ', 'ZTQLEV', 'ZTQLEV_bnds', 'TTZ'])
4 is not ok 
dict_keys(['X21_1001', 'Y21_701', 'ZUVLEV', 'ZUVLEV_bnds', 'TIME1', 'UUZ', 'VVZ', 'ZTQLEV', 'ZTQLEV_bnds', 'TTZ'])
6 is not ok 
dict_keys(['X21_100', 'Y21_70', 'ZUVLEV', 'ZUVLEV_bnds', 'TIME2', 'UUZ', 'VVZ', 'ZTQLEV', 'ZTQLEV_bnds', 'TTZ'])
8 is not ok 
dict_keys(['X21_100', 'Y21_70', 'ZUVLEV', 'ZUVLEV_bnds', 'TIME2', 'UUZ', 'VVZ', 'ZTQLEV', 'ZTQLEV_bnds', 'TTZ'])


In [34]:
def get_wind_time(netCDF_file_path, location):
    """
    args:
    -----
        netCDF_file_path: String
                          netCDF path file containing time series of size (l,)
        location: tuple (k,j,i)
                  indices given in ferret indices (starting from 1)
    return:
    -------
        uuz: float numpy array of size (l,)
             wind speed according to x axis in (m/s)
        vvz: float numpy array (l,)
             wind speed according to y axis in (m/s)
        time: float numpy array (l,)
              minutes elasped from a given date start_time
        start_time: String
                    ISO date time
        
    """
    with netcdf.netcdf_file(netCDF_file_path, mode='r', mmap=False) as f:
        # Attention use "- 1" due to the shift between ferret and python
        # k is a special case sometimes 3 levels sometimes 2
        k = f.variables['UUZ'][0,:,0,0].shape[0] - 1 # 80 m 
        j = location[1] - 1 # j wind farm
        i = location[2] - 1 # i wind farm
        
        # get time series in numpy array
        uuz = f.variables['UUZ'][:,k,j,i].copy()
        vvz = f.variables['VVZ'][:,k,j,i].copy()
        
        time = f.variables['TIME'][:].copy()
        
        # Get only the ISO date from "minutes since YYYY-MM-DD HH:mm:ss"
        # + convert from bytes to String
        start_time = f.variables['TIME'].units.decode()[14:]
        
        f.close()
    
    return uuz, vvz, time, start_time

In [35]:
path_mar_data = "data/MAR/concat.nc"

In [36]:
locations = [(2,31,61), (2,17,27), (2,23,39)] # (k,j,i)
    
# Init list to contain all the data
dataset = []

for i, location in enumerate(locations):
    print("(i, location) = ({}, {})".format(i, location))
    # time_serie = prod_per_wind_farm.iloc[i] # Get prod for i th wind farm

    uuz, vvz, time, start_time = get_wind_time(netCDF_file_path=\
                                                            path_mar_data, 
                                               location=location)

    time = [convert_min_to_date(start_time, elapsed_minutes) 
            for elapsed_minutes in time]

    # DataFrame
    data = {"uuz":uuz, "vvz": vvz, "time":time}
    netcdf_df = pd.DataFrame(data)

    # Interpolation part
    # Wind Speed X
    points = [convert_date_to_min(date=str(date)) \
                for date in netcdf_df['time'][:]]
    values = netcdf_df['uuz'][:]

    interp_x = [convert_date_to_min(date=date[:-1]) 
                for date in time_serie.index.values]

    interp_speedX = np.interp(interp_x, points, values)

    # Wind Speed Y
    points = [convert_date_to_min(date=str(date)) \
                for date in netcdf_df['time'][:]]
    values = netcdf_df['vvz'][:]

    interp_y = [convert_date_to_min(date=date[:-1]) 
                for date in time_serie.index.values]

    interp_speedY = np.interp(interp_y, points, values)


    # Norm Wind Speed
    norm_ws = np.sqrt(np.square(interp_speedX) + np.square(interp_speedY))

    # Angle Wind Speed
    angle_ws = np.arctan2(interp_speedY, interp_speedX)

    # Dataset Creation
    dataset = dataset + [("prod_wf{}".format(i), 
                          time_serie.values / power_installed[i]), 
                         ("windSpeedNorm{}".format(i), norm_ws),
                         ("windSpeedAngle{}".format(i), angle_ws)]

dataset = dataset + [("time", time_serie.index.values)]

# convert list into dictionary to construct the DataFrame   
dataset = {key:value for key, value in dataset}

df = pd.DataFrame(dataset)

(i, location) = (0, (2, 31, 61))


OSError: [Errno 22] Invalid argument

In [54]:
netCDF_file_path = "data/MAR/concat.nc"
with netcdf.netcdf_file(netCDF_file_path, mode='r') as f:
        # Attention use "- 1" due to the shift between ferret and python
        # k is a special case sometimes 3 levels sometimes 2
        k = f.variables['UUZ'][0,:,0,0].shape[0] - 1 # 80 m 
        j = location[1] - 1 # j wind farm
        i = location[2] - 1 # i wind farm
        
        # get time series in numpy array
        uuz = f.variables['UUZ'][:,k,j,i].copy()
        vvz = f.variables['VVZ'][:,k,j,i].copy()
        
        time = f.variables['TIME'][:].copy()
        
        # Get only the ISO date from "minutes since YYYY-MM-DD HH:mm:ss"
        # + convert from bytes to String
        start_time = f.variables['TIME'].units.decode()[14:]
        
        f.close()

ValueError: cannot reshape array of size 0 into shape (35040,2,50,80)

In [57]:
with netcdf.netcdf_file("data/MAR/concat.nc", mode='r') as file:
    print("hello")

ValueError: cannot reshape array of size 0 into shape (35040,2,50,80)